<a href="https://colab.research.google.com/github/OxanaMakhneva/test/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22stratified_group_k_fold_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import chisquare, chi2_contingency
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

#Расчет среднего для списка расстояний Пирсона между рапределениями таргета во всей выборке и распределнимя при помещении
#группы с конкретным лэйблом в каждый из возможных фолдов
def calc_chi2_per_fold(target_distrib, uniq_strat_labels, actual_target_distrib_per_fold, distrib_per_group, fold_ind, K,\
                       possible_target_distrib_per_fold):
    #Временно добавляем к текущему надору меток заданного фолда распределение заданной метки
    actual_target_distrib_per_fold[fold_ind] = \
    merge_dicts(actual_target_distrib_per_fold[fold_ind], distrib_per_group)
    #Для каждого фолда считаем кси
    #Список для дисперсий по каждому из значений таргета
    chi_per_folds = []
    for ind in range(K):
        contingency_table = \
        pd.DataFrame({target:[target_distrib[target], \
                              actual_target_distrib_per_fold[ind][target]] for target in target_distrib.keys()})
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        chi_per_folds.append(chi2)
    possible_target_distrib_per_fold[fold_ind] = np.mean(chi_per_folds)

#Объединяет два словаря, складывая значения словарей друг с другом по одному ключу
def merge_dicts(dict1, dict2):
    return {key: dict1.get(key, 0) + dict2.get(key, 0) for key in set(dict1) | set(dict2)}

def parallel_calc_chi2_per_fold(target_distrib, uniq_strat_labels, target_distrib_per_fold, distrib_per_group, K):
    global possible_target_distrib_per_fold
    possible_target_distrib_per_fold = {}
    args = []
    with ThreadPoolExecutor(K) as executor:
        executor.map(calc_chi2_per_fold, [target_distrib]*K, [uniq_strat_labels]*K, \
                                         [target_distrib_per_fold]*K, [distrib_per_group]*K,
                                         list(range(K)),
                                         [K]*K,
                                         [possible_target_distrib_per_fold]*K)

    return possible_target_distrib_per_fold


'''
Разделяет выборку на К групп, каждая из которых состоит из К частей, где
- одна часть - предполагается для использования в тесте
- K-1 часть - предполагается для использования в обучении
При этом записи в части подбираются так, чтобы
- одновременно не было записей с одинаковыми метками И в тесте И в трейне
- распределение целевой переменной было примерно одинаковым в общей выборке, в тесте и в трейне
На вход подается:
df - ДФ
y_col - название столбца с таргетом
label_col - название столбца с меткой
K - число фолдов
'''
def stratified_group_k_fold(df, y_col, label_col, K):
    y = df[y_col].values
    strat_labels = df[label_col].values

    #Сколько всего значений в выборке
    df_len = len(y)
    #Список уникальных меток групп
    uniq_strat_labels = list(set(strat_labels))
    uniq_target = list(set(y))
    #Проверяем, что число уникальных меток больше или равно числу фолдов
    if len(uniq_strat_labels) < K:
        raise Exception('Number of unique labels less then K')
    print('1')
    #Cловарь для подсчета распределения таргета
    target_distrib = {target: 0 for target in set(y)}
    print('11')
    #Словарь словарей для фиксации распределения таргета внутри каждой группы меток стратификации
    target_distrib_per_group = {label: {target: 0 for target in uniq_target} for label in uniq_strat_labels}
    print('2')
    #Заполнение словарей распределения таргета в выборке и распределения таргета по группам
    for pos in range(df_len):
        target_distrib[y[pos]] += 1
        target_distrib_per_group[strat_labels[pos]][y[pos]] = target_distrib_per_group[strat_labels[pos]][y[pos]] + 1
    print('3')
    #Словарь словарей для фиксации распределения таргета внутри каждого фолда с учетом помещенных в него групп меток
    target_distrib_per_fold = {fold_ind: {target: 0 for target in y} for fold_ind in range(K)}
    #Словарь для итговой расстасофки меток групп по фолдам
    strat_labels_per_fold = {fold_ind: [] for fold_ind in range(K)}
    print('4')
    #Сортируем словарь по расстоянию к-си квадрат от основного распределения
    #Метки, самые близкие к генеральному распределению, первыми поместим в фолды
    chi2_dict = {}
    for label in uniq_strat_labels:
        #Формируем таблицу сопряжений для каждой метки (!можно сделать параллельно)
        #(в ней первая строка - распределение в полной выборке, вторая строка - распределение только по одной группе)
        contingency_table = \
        pd.DataFrame({target:[target_distrib[target], target_distrib_per_group[label][target]] for target in target_distrib.keys()})
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        chi2_dict[label] = chi2
    #Отбираем k лэйблов с минимальным chi2
    chi2_turpl = sorted(chi2_dict.items(), key=lambda items: items[1])
    chi2_turpl = chi2_turpl[:K]
    inital_labels = [pos[0] for pos in chi2_turpl]

    #Первые к лэйблов с минимальным расстоянием от исходного расспределения записываем в фолды (1 группа лэйблов в 1 фолд)
    for ind in range(K):
        #метка, которую помещаем в фолд
        label = inital_labels[ind]
        #фиксируем в словаре
        strat_labels_per_fold[ind] = [label]
        #распредление, которое соответствует метке
        target_distrib_per_fold[ind] = target_distrib_per_group.pop(label)

    #Итеририруем по оставшимся меткам групп и формируем словарь {фолд: [подобранные метки]}
    for strat_label, distrib_per_group in tqdm(target_distrib_per_group.items()):
        #Для каждого фолда рассчитываем дисперсию для случая, когда в фолд (fold_ind) будет помещено
        #распределение группы меток (distrib_per_group)
        possible_target_distrib_per_fold = \
        parallel_calc_chi2_per_fold(target_distrib, uniq_strat_labels, target_distrib_per_fold, distrib_per_group, K)
#         possible_target_distrib_per_fold = \
#         {fold_ind: calc_chi2_per_fold(target_distrib, uniq_strat_labels, target_distrib_per_fold, distrib_per_group, fold_ind, k) for fold_ind in range(k)}
        #Отбираем тот фолд, у которого дисперсия минимальна
        best_fold_ind = min(possible_target_distrib_per_fold, key = possible_target_distrib_per_fold.get)
        #Помещаем в словарь распределения по фолдам к ключу отобранного фолда распределение текущей метки
        target_distrib_per_fold[best_fold_ind] = merge_dicts(target_distrib_per_fold[best_fold_ind], distrib_per_group)
        #Помещаем в словарь со списком меток для каждого фолда к отрбранному форлду отработанную метку
        calc_strat_labels = strat_labels_per_fold[best_fold_ind]
        calc_strat_labels.append(strat_label)
        strat_labels_per_fold[best_fold_ind] = calc_strat_labels

    #Формируем итератор, который будет выдавать индексы для трэйна и для теста
    #Для каждого фолда один помещаем в тест, остальные - в трейн. Число комбинаций = числу фолдов.
    #Отношение теста к трейну ~ 1/(k-1)
    for fold_ind, labels_in_fold in strat_labels_per_fold.items():
        #Расчитываем метки, которые попадут в трэйн выборку
        train_labels = list(set(uniq_strat_labels) - set(labels_in_fold))
        #Рассчитываем индексы, которые соответсвуют отобранным меткам
        train_indexes = df.query(f'{label_col} in @train_labels').index.to_list()
        #Все индексы, которые не попали в трэйн отправдяем в тест
        test_indexes = df.query(f'{label_col} not in @train_labels').index.to_list()
        yield train_indexes, test_indexes
